In [1]:
from PIL import Image
import numpy as np
from tqdm import tqdm
import random

In [2]:
img = np.array(Image.open('image.png').convert('RGBA'))

In [3]:
w, h, _ = img.shape

In [4]:
k = 150 # tajny klucz generatora pseudolosowego
n = 30 # liczba naturalna n
d = 5 # delta

In [5]:
def random_points(prng, w, h, n, e={}):
    result = []
    existing = e
    for i in range(n):
        new_point = (prng.randint(0, w - 1), prng.randint(0, h - 1))
        while new_point in existing:
            new_point = (prng.randint(0, w - 1), prng.randint(0, h - 1))
        existing[new_point] = True
        result.append(new_point)
    return result, existing

In [6]:
def change(array, delta):
    for i in range(len(array) - 1):
        array[i] = min(max(array[i] + delta, 0), 255)
    return array

# Osadzenie znaku wodnego

In [7]:
def embed_watermark(mat, k, n, w, h):
    prng = random.Random(k)
    A, e1 = random_points(prng, w, h, n, {})
    B, _ = random_points(prng, w, h, n, e1)
    for (ax, ay), (bx, by) in zip(A, B):
        mat[ax][ay] = change(mat[ax][ay], d)
        mat[bx][by] = change(mat[bx][by], -d)
    Sn = 2*d*n + sum([(mat[ax][ay].astype(dtype=np.int32) - mat[bx][by].astype(dtype=np.int32)).sum() for (ax, ay), (bx, by) in zip(A, B)])
    return Sn, mat

In [8]:
Sn, img_with_watermark = embed_watermark(img, k, n, w, h)

In [9]:
Sn

23

In [10]:
img_with_watermark2 = Image.fromarray(img_with_watermark, 'RGBA')
img_with_watermark2.save("image with watermark.png")

# Detekcja znaku wodnego

In [11]:
def detect_watermark(mat, k, n, w, h):
    prng2 = random.Random(k)
    Ap, e2 = random_points(prng2, w, h, n, {})
    Bp, _ = random_points(prng2, w, h, n, e2)
    for (ax, ay), (bx, by) in zip(Ap, Bp):
        mat[ax][ay] = change(mat[ax][ay], -d)
        mat[bx][by] = change(mat[bx][by], d)
    Sn = sum([(mat[ax][ay].astype(dtype=np.int32) - mat[bx][by].astype(dtype=np.int32)).sum() for (ax, ay), (bx, by) in zip(Ap, Bp)])
    return Sn

In [12]:
detect_watermark(img_with_watermark, k, n, w, h)

-1177

# Testy na ataki

In [13]:
img2 = np.array(Image.open('image with watermark.png').convert('RGBA'))

In [14]:
np.array_equal(img, img2)

False

In [15]:
Sn = detect_watermark(img2, k, n, w, h)

In [16]:
Sn

-1177

In [17]:
img3 = np.array(Image.open('image with watermark jpeg.jpg').convert('RGBA'))

In [18]:
Sn = detect_watermark(img3, k, n, w, h)
Sn

-1491

In [19]:
img4 = np.array(Image.open('image with watermark gif.gif').convert('RGBA'))

In [20]:
Sn = detect_watermark(img4, k, n, w, h)
Sn

-1206

In [21]:
img5 = np.array(Image.open('image with watermark scaled.png').convert('RGBA'))

In [22]:
Sn = detect_watermark(img5, k, n, w, h)
Sn

-1177

In [23]:
img6 = np.array(Image.open('image with watermark changed color depth.png').convert('RGBA'))

In [24]:
Sn = detect_watermark(img6, k, n, w, h)
Sn

-1136

In [25]:
img7 = np.array(Image.open('image with watermark bmp to gif.gif').convert('RGBA'))
Sn = detect_watermark(img7, k, n, w, h)
Sn

-1087

In [26]:
img8 = np.array(Image.open('image with watermark modified.png').convert('RGBA'))
Sn = detect_watermark(img8, k, n, w, h)
Sn

-776